Django have a buildin user system. Let's dive in the user system first, then we going to integrate into the blog system that we build. We going to access user object on django shell with

In [ ]:
python manage.py shell

In [ ]:
from django.contrib.auth.models import User

First thing is, django user is stored in database. It have their own set of method. Now let's create a user.

In [ ]:
user = User.objects.create_user("john", "john.doe@example.com", "randompasswd")

you can access a user object like how you access user model. Notice the create_user is a special method for django user. 

In [ ]:
user = User.objects.get(username="john")

In [ ]:
user.change_password("icantdothatdave")

In [ ]:
user.save()

User also have concept of groups, permissions etc. We won't be using it here, it is useful for complex project.

Now we have models time to use it in program. 

In [ ]:
from django.contrib.auth import authenticate

In [ ]:
user = authenticated(username="john", password="icantdothatdave")

Now check if user is authenticated

In [ ]:
if user.is_authenticated:
    print("logged in")
else:
    print("not logged in")

Now lets use it to a view, we mostly do following as an example, I will show you how we usually do it later. Let's do a few setup.

In [ ]:
python manage.py startapp sillylogin

Now create some views

In [ ]:
from django.views import View
from django.shortcuts import render
from django.shortcuts import redirect
from django.contrib.auth.models import User
from django.contrib.auth import authenticate
from django.contrib.auth import login
from django.contrib.auth import logout


class LoginView(View):
    def get(self, request):
        return render(request, "login_form.html")

    def post(self, request):
        form_data = request.POST
        username = form_data.get("username")
        password = form_data.get("password")

        user = authenticate(username=username, password=password)

        if user:
            login(request, user)
            return redirect("silly-loggedin-view")
        return render(request, "login_error.html")


class LoggedInView(View):
    def get(self, request):
        if request.user.is_authenticated:
            return render(request, "loggedin_view.html", {"status": "logged in"})
        return render(request, "loggedin_view.html", {"status": "not login"})


class RegistrationView(View):
    def get(self, request):
        return render(request, "registration_form.html")

    def post(self, request):
        form_data = request.POST
        username = form_data.get("username")
        password = form_data.get("password")
        email = form_data.get("email")
        try:
            user = User.objects.create_user(username, email, password)
            return redirect("silly-login-view")
        except Except as e:
            s = str(e)
            return render(request, "registration_form.html", {"error_msg", s})


class LogoutView(View):
    def get(self, request):
        return redirect("silly-login-view")


Before we go on let's look at vew code. Look at LoggedInView. Notice there is a request.user. It is effectively user stored in request after you login. Sometime it just stored as anonymmouse user if not. I uses session feature in browser. 

Also notice that user object is use like model. Because it is but with some custom methods. 

Now create some template, first create sillylogin/templates/login_form.html. We will not use template inheritance here. 

In [ ]:
<html>
<head><title>Login form</title></head>
<body>
    <form action="" method="POST">{% csrf_token %}
        <p><label for="username">Username:</label><input type="text" name="username"></p>
        <p><label for="password">Password:</label><input type="password" name="password"></p>
        <p><input type="submit" value="login"></p>
    </form>
</body>
</html>

Now create a sillylogin/templates/loggedin_view.html.

In [ ]:
<html>
<head><title>Login form</title></head>
<body>
    <p>{% if request.user.is_authenticated %}Welcome {{ request.user.username}}{% else %}Hello stranger{% endif %}</p>
    <p>{{status}}</p>
</body>
</html>


Notice that request.user is pass to template, it is enabled by default. You can check for user in templates

Now create a sillylogin/templates/registration_form.html

In [ ]:
<html>
<head><title>Register form</title></head>
<body>
    <p>{{error_msg}}</p>
    <form action="" method="POST">{% csrf_token %}
        <p><label for="username">Username:</label><input type="text" name="username"></p>
        <p><label for="password">Password:</label><input type="password" name="password"></p>
        <p><label for="email">Email:</label><input type="text" name="email"></p>
        <p><input type="submit" value="register"></p>
    </form>
</body>
</html>


Now have all the sillylogin/urls.py ready

In [ ]:
from django.conf.urls import url
from sillylogin.views import *

urlpatterns = [
    url(r"^login", LoginView.as_view(), name="silly-login-view"),
    url(r"^logout", LogoutView.as_view(), name="silly-logout-view"),
    url(r"^register", RegistrationView.as_view(), name="silly-registration-view"),
    url(r"^$", LoggedInView.as_view(), name="silly-loggedin-view"),
]


Now add a line to codingshophouse_blog/urls.py so that it look like

In [ ]:
from django.conf.urls import url
from django.conf.urls import include
from django.contrib import admin

urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r"^blog/", include("blog.urls")),
    url(r"^sillylogin/", include("sillylogin.urls")),
]


Then add a line to settings.py INSTALLED_APP section

In [ ]:
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'blog',
    'sillylogin',
]

The above code is mostly to show how it work behind the scene. In reality django have a lot of short cut that make everything easy. But as usual setup needed.

First create a template for login. Django auth models already have form defined, so we won't be needing to create a new form. First create a new directory

In [ ]:
mkdir templates/registration

Then add a template. template/registration/login.html

In [ ]:
{% extends 'base.html' %}

{% block title %}Login{% endblock %}

{% block content %}
  <h2>Login</h2>
  <form method="post">
    {% csrf_token %}
    {{ form.as_p }}
    <button type="submit">Login</button>
  </form>
{% endblock %}

Now create a template template/registration/logout.html

In [ ]:
% extends 'base.html' %}

{% block title %}Logged out{% endblock %}

{% block content %}
  <p>Logout</p>
{% endblock %}

Now update the urls.py

In [ ]:
from django.conf.urls import url
from django.conf.urls import include
from django.contrib import admin
from django.contrib.auth import views as auth_views

urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r"^blog/", include("blog.urls")),
    url(r"^sillylogin/", include("sillylogin.urls")),
    url(r"^login/", auth_views.login_views),
    url(r"^logout/", auth_views.logout_views),
]


To actually use it you use a function called login_required. 2 way to tell a view require login. 1 way is to do at urls. The other is do it at class. Let do it in class

Django provide afew thing like change password etc. I won't show here. It does not provide a registration view, that's where you can use django registration. There is a few, there is django registration and django registration redux, they do the same thing, but do it different ways. I don't know which is better, but I like django registration

In [ ]:
pip install django_registration

Now create a template, template/registration/registration_form.html

In [ ]:
{% extends 'base.html' %}

{% block title %}Registration{% endblock %}

{% block content %}
  <form method="post">
    {% csrf_token %}
    {{ form.as_p }}
    <button type="submit">Login</button>
  </form>
{% endblock %}


Now add urls

In [ ]:
from django.conf.urls import url
from django.conf.urls import include
from django.contrib import admin
from django.contrib.auth import views as auth_views


urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r"^blog/", include("blog.urls")),
    url(r"^sillylogin/", include("sillylogin.urls")),
    url(r"^login/", auth_views.login),
    url(r"^logout/", auth_views.logout),
    url(r'^accounts/', include('registration.backends.simple.urls')),
]


The registration will redirect to non-existence page. And I am lazy to change the success url, it require writing more code. So let's clean up and redo the urls shall we

In [ ]:
from django.conf.urls import url
from django.conf.urls import include
from django.contrib import admin
from django.contrib.auth import views as auth_views


urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r'^accounts/', include('registration.backends.simple.urls')),
    url(r"", include("blog.urls")),
]

Other than I lazy to do password reset page, we are pretty much done!